In [1]:
%env WANDB_DISABLED=1

env: WANDB_DISABLED=1


In [2]:
import transformers
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_metric

def finetune_task(glue_task, model_name_or_path, tokenizer_name_or_path, output_folder, batch_size = 16):
    """
    Inputs:
    |  glue_task = A single GLUE task to finetune towards
    |  model_name_or_path = model name (hugging face) or path (local)
    |  tokenizer_name_or_path = model name (hugging face) or path (local)
    |  sub_folder = path to save in lucky_workspace [NEED REVAMP]
    |  batch_size = batch size during fine-tuning. Default to 16.
    Outputs:
    |  Move all relevant data to working directory (Currently only moves to lucky_workspace) [NEED REVAMP]
    Extra:
    |  glue_task = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
    """
    
    # Helper Functions
    def preprocess_function(examples):
        sentence1_key, sentence2_key = task_to_keys[task]
        if sentence2_key is None:
            return tokenizer(examples[sentence1_key], truncation=True)
        return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)
    
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        if task != "stsb":
            predictions = np.argmax(predictions, axis=1)
        else:
            predictions = predictions[:, 0]
        return metric.compute(predictions=predictions, references=labels)
    
    print("Getting Task...")
    task = glue_task
    model_checkpoint = model_name_or_path #"/mnt/data1/lucky_workspace/bert-base-uncased-cl/checkpoint-53000"
    tokenizer_checkpoint = tokenizer_name_or_path #"bert-base-uncased"
    batch_size = batch_size
    
    
    print("Getting Metric...")
    actual_task = "mnli" if task == "mnli-mm" else task
    dataset = load_dataset("glue", actual_task)
    metric = load_metric('glue', actual_task)
    
    print("Getting Tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint, use_fast=True)
    
    task_to_keys = {
        "cola": ("sentence", None),
        "mnli": ("premise", "hypothesis"),
        "mnli-mm": ("premise", "hypothesis"),
        "mrpc": ("sentence1", "sentence2"),
        "qnli": ("question", "sentence"),
        "qqp": ("question1", "question2"),
        "rte": ("sentence1", "sentence2"),
        "sst2": ("sentence", None),
        "stsb": ("sentence1", "sentence2"),
        "wnli": ("sentence1", "sentence2"),
    }
    
    print("Encoding...")
    encoded_dataset = dataset.map(preprocess_function, batched=True)
    
    print("Setting Labels...")
    num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    
    metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
    model_name = model_checkpoint.split("/")[-1]

    print("Setting Training Arguments...")
    args = TrainingArguments(
        f"{output_folder}/{model_name}-finetuned-{task}",
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=3,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model=metric_name
    )
    
    validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
    trainer = Trainer(
        model,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset[validation_key],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    print("Training and Evaluation...")
    trainer.train()
    trainer.evaluate()

In [5]:
model_name_or_local_path = "models/bert-base-uncased-1p-no_cl/bert-base-uncased-1p-no_cl-checkpoint53000"
tokenizer_name_or_local_path = "bert-base-uncased"
output_folder = "models/bert-base-uncased-cl-1p-no_cl-finetune"

In [6]:
#  glue_task = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
fast_glue_task = ["cola", "mrpc", "rte", "sst2", "stsb", "wnli"]

for task in fast_glue_task:
    finetune_task(task, model_name_or_local_path, tokenizer_name_or_local_path, output_folder)

Getting Task...
Getting Metric...


Reusing dataset glue (/root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Getting Tokenizer...


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/ma

...

Encoding...
Setting Labels...


Some weights of the model checkpoint at models/bert-base-uncased-1p-no_cl/bert-base-uncased-1p-no_cl-checkpoint53000 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initializ

Setting Training Arguments...
Training and Evaluation...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.685110,0.563380
2,No log,0.696131,0.436620
3,No log,0.690771,0.563380


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx. If sentence1, sentence2, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 71
  Batch size = 16
Saving model checkpoint to models/bert-base-uncased-cl-1p-no_cl-finetune/bert-base-uncased-1p-no_cl-checkpoint53000-finetuned-wnli/checkpoint-40
Configuration saved in models/bert-base-uncased-cl-1p-no_cl-finetune/bert-base-uncased-1p-no_cl-checkpoint53000-finetuned-wnli/checkpoint-40/config.json
Model weights saved in models/bert-base-uncased-cl-1p-no_cl-finetune/bert-base-uncased-1p-no_cl-checkpoint53000-finetuned-wnli/checkpoint-40/pytorch_model.bin
tokenizer config file saved in models/bert-base-uncased-cl-1p-no_cl-finetune/bert-base-uncased-1p-no_cl-checkpoint53000-finetuned-wnli/checkpoint-40/tokenizer_config.json
Sp

In [7]:
# slow_glue_task = ["mnli", "mnli-mm", "qnli", "qqp"]
# for task in slow_glue_task:
#     finetune_task(task, model_name_or_local_path, tokenizer_name_or_local_path, output_folder)

In [11]:
import torch
torch.cuda.empty_cache()